In [1]:
# Importing all the dependencies
import pandas as pd
import numpy as np
import os
import warnings
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Font
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles.alignment import Alignment

In [2]:
###Input Files
##This is the configuraiton and the only thing to change in this code whenever you rerun
cwd = os.getcwd()
demand_output = "28-04-2024_Demand_Automation.xlsx"
capacity_output = "Capacity_DD_Filtered.xlsx"
production_output = "03-05-2024_Prod_Automation.xlsx"
freight_raw = os.path.join(cwd, "client_data", "Freight.xlsx")
country_mapping_file =os.path.join(cwd, "client_data", "demand_country_mapping.xlsx")


# Production
production_output = "03-05-2024_Prod_Automation.xlsx"

In [3]:
#######Getting all combinations for primary freight
demand_df = pd.read_excel(demand_output,sheet_name = 'Demand_output')
demand_df = demand_df[demand_df['Ret_Flag']==1]

factories = ['NFC', 'GFC', 'KFC']
warehouses = ['NFCM', 'GFCM', 'KFCM']


# Map factory to country
factory_country_map = {
    'NFC': 'Saudi Arabia',
    'GFC': 'United Arab Emirates',
    'KFC': 'Kuwait'
}

###Creating all possible combinations of factory and warehouse
warehouse_countries = {
    'NFCM': 'Saudi Arabia',
    'GFCM': 'United Arab Emirates',
    'KFCM': 'Kuwait'
}

skus = demand_df['FGSKUCode'].unique()

print("Total Unique SKUs",len(skus))
print("Possible Combinations",len(skus)*3*3)

# Create all combinations of SKU, Factory, and Warehouse
combinations = pd.MultiIndex.from_product([skus, factories, warehouses], names=['FGSKUCode', 'Factory', 'Warehouse']).to_frame(index=False)

print("Possible Combinations generated for Primary Freight - ",len(combinations))

# Merge the combinations with the demand dataframe to keep the Ret_Flag column
result_df = pd.merge(combinations, demand_df[['FGSKUCode', 'Ret_Flag']].drop_duplicates(), on='FGSKUCode', how='left')
result_df['Destination Country'] = result_df['Warehouse'].map(warehouse_countries)
result_df['Origin Country'] = result_df['Factory'].map(factory_country_map)

print("Possible Combinations generated for Result Df - ",len(result_df))

Total Unique SKUs 551
Possible Combinations 4959
Possible Combinations generated for Primary Freight -  4959
Possible Combinations generated for Result Df -  4959


In [4]:
#######Preparing master freight data and calculate averages for imputation
freight_df = pd.read_excel(freight_raw, sheet_name='II. Template - FreightRawData', skiprows=3)
# Drop the first blank column
freight_df = freight_df.drop(columns=['Unnamed: 0'])
freight_df.columns = ['FG Code', 'FG Desc', 'UOM', 'Origin', 'Destination', 'truckload', 'freight_per_truck', 'customs', 'freight_per_uom', 'retain_flag']


# Convert relevant columns to numeric types and calculate cost per carton
freight_df['truckload'] = pd.to_numeric(freight_df['truckload'], errors='coerce')
freight_df['freight_per_truck'] = pd.to_numeric(freight_df['freight_per_truck'], errors='coerce')
freight_df['Total Cost(FRT_CD)'] = freight_df['freight_per_truck'] / freight_df['truckload']

# Add origin country and destination country columns
freight_df['Origin Country'] = freight_df['Origin'].map(factory_country_map)
freight_df['Destination Country'] = freight_df['Destination']

In [5]:
print("Number of rows in freight df - ", len(freight_df))

Number of rows in freight df -  24732


In [6]:
# Calculate average cost per carton between countries for missing value imputation
average_cost_df = freight_df.groupby(['Origin Country', 'Destination Country'])['Total Cost(FRT_CD)'].mean().reset_index()
# Calculate average cost per carton for each destination
destination_avg_cost_df = freight_df.groupby('Destination Country')['Total Cost(FRT_CD)'].mean().reset_index()

# Filter out NaN values before creating dictionaries
average_cost_df = average_cost_df.dropna(subset=['Total Cost(FRT_CD)'])
destination_avg_cost_df = destination_avg_cost_df.dropna(subset=['Total Cost(FRT_CD)'])


# Calculate the maximum of the average destination costs
average_cost_dict = average_cost_df.set_index(['Origin Country', 'Destination Country'])['Total Cost(FRT_CD)'].to_dict()
destination_avg_cost_dict = destination_avg_cost_df.set_index('Destination Country')['Total Cost(FRT_CD)'].to_dict()
# Calculate the maximum of the average destination costs
##THis is a single value 
max_destination_avg_cost = destination_avg_cost_df['Total Cost(FRT_CD)'].max() 


# Debugging: Print the values being used for imputation
def replace_with_average(row):
    if row['Origin Country'] == row['Destination Country']:
        return 0
    if pd.isna(row['Total Cost(FRT_CD)']):
        origin_dest_cost = average_cost_dict.get((row['Origin Country'], row['Destination Country']))
        dest_avg_cost = destination_avg_cost_dict.get(row['Destination Country'])
        if origin_dest_cost is None and dest_avg_cost is None:
            print(f"No data for Origin: {row['Origin Country']}, Destination: {row['Destination Country']}. Using max cost: {max_destination_avg_cost}")
        else:
            print(f"Origin: {row['Origin Country']}, Destination: {row['Destination Country']}, Origin-Dest Avg Cost: {origin_dest_cost}, Dest Avg Cost: {dest_avg_cost}")
        if origin_dest_cost is not None:
            return origin_dest_cost
        if dest_avg_cost is not None:
            return dest_avg_cost
        return max_destination_avg_cost
    return row['Total Cost(FRT_CD)']

In [7]:
# Perform a left join using result_df on the left and add the Total Cost(FRT_CD) from the freight_df
result_df = pd.merge(result_df, freight_df[['FG Code', 'Origin Country', 'Destination Country', 'Total Cost(FRT_CD)']], left_on=['FGSKUCode', 'Origin Country', 'Destination Country'], right_on=['FG Code', 'Origin Country', 'Destination Country'], how='left')
# Treat empty strings in 'Total Cost(FRT_CD)' as NaN
result_df['Total Cost(FRT_CD)'] = result_df['Total Cost(FRT_CD)'].replace('', np.nan)
result_df['Missing_Flag'] = result_df['Total Cost(FRT_CD)'].isnull().astype(int)

In [8]:
# Perform a left join using result_df on the left and add the Total Cost(FRT_CD) from the freight_df
result_df['Total Cost(FRT_CD)'] = result_df.apply(replace_with_average, axis=1)

Origin: United Arab Emirates, Destination: Saudi Arabia, Origin-Dest Avg Cost: 1.6639302043710233, Dest Avg Cost: 0.39838039472201886
Origin: United Arab Emirates, Destination: Kuwait, Origin-Dest Avg Cost: 1.2622971142565018, Dest Avg Cost: 0.5719216205990153
Origin: United Arab Emirates, Destination: Saudi Arabia, Origin-Dest Avg Cost: 1.6639302043710233, Dest Avg Cost: 0.39838039472201886
Origin: United Arab Emirates, Destination: Kuwait, Origin-Dest Avg Cost: 1.2622971142565018, Dest Avg Cost: 0.5719216205990153
Origin: United Arab Emirates, Destination: Saudi Arabia, Origin-Dest Avg Cost: 1.6639302043710233, Dest Avg Cost: 0.39838039472201886
Origin: United Arab Emirates, Destination: Kuwait, Origin-Dest Avg Cost: 1.2622971142565018, Dest Avg Cost: 0.5719216205990153
Origin: United Arab Emirates, Destination: Saudi Arabia, Origin-Dest Avg Cost: 1.6639302043710233, Dest Avg Cost: 0.39838039472201886
Origin: United Arab Emirates, Destination: Kuwait, Origin-Dest Avg Cost: 1.26229711

In [9]:
result_df.shape

(4959, 9)

In [10]:
average_cost_df.to_excel("Average_Freight.xlsx")
destination_avg_cost_df.to_excel("Average_Destination_Freight.xlsx")
result_df.to_excel("Imputed_primary_freight_data.xlsx")

In [11]:
production_df=pd.read_excel(production_output,sheet_name='Production_output',skiprows=1)
production_df = production_df[production_df['retain_flag']==1]

df_output1 = pd.merge(production_df, result_df, left_on=['FactCode', 'Unified code'], right_on=['Factory', 'FGSKUCode'], how='left')
df_output1=df_output1[['Factory', 'Warehouse', 'FGSKUCode', 'Total Cost(FRT_CD)','retain_flag', 'Missing_Flag']]
df_output1.columns = ['Origin', 'WHCode', 'FG Code', 'Total Cost(FRT_CD)', 'retain_flag', 'Missing_Flag']
df_output1=df_output1.drop_duplicates()

df_output1.head()

,Origin,WHCode,FG Code,Total Cost(FRT_CD),retain_flag,Missing_Flag
0,NFC,NFCM,4001351001,0.000000,1,0
1,NFC,GFCM,4001351001,0.667965,1,0
2,NFC,KFCM,4001351001,0.646465,1,0
3,KFC,NFCM,4001351001,0.507305,1,0
4,KFC,GFCM,4001351001,0.461495,1,0


In [12]:
df_output1.shape

(2121, 6)

In [13]:
# QC Output for missing values
distinct_counts = df_output1.nunique()
null_counts = df_output1.isnull().sum()
counts = df_output1.count()
summary_stats = df_output1.describe(percentiles=[0.25, 0.5, 0.75]).transpose()

result_df1 = pd.DataFrame({
    'Count': counts,
    'Distinct Values': distinct_counts,
    'Null Count': null_counts,
    'Mean': summary_stats['mean'],
    'Std': summary_stats['std'],
    'Minimum Value': summary_stats['min'],
    'Percentile Distribution (25%)': summary_stats['25%'],
    'Percentile Distribution (50%)': summary_stats['50%'],
    'Percentile Distribution (75%)': summary_stats['75%'],
    'Maximum Value': summary_stats['max']
})

result_df1['Column Name'] = result_df1.index
result_df1 = result_df1[['Column Name', 'Count', 'Distinct Values', 
                         'Null Count', 'Mean', 'Std', 'Minimum Value', 
                         'Percentile Distribution (25%)', 'Percentile Distribution (50%)', 
                         'Percentile Distribution (75%)', 'Maximum Value']]

                        
# Generate result2
filtered_df = df_output1[df_output1['Total Cost(FRT_CD)'] == 0]
grouped_df = filtered_df.groupby('FG Code').agg({
    'Origin': list,
    'Total Cost(FRT_CD)': 'first'
}).reset_index()
grouped_df['Origin'] = grouped_df['Origin'].apply(lambda x: ', '.join(x))
grouped_df.columns = ['FG Code', 'Origin', 'Total Cost(FRT_CD)']
result2 = grouped_df[['FG Code', 'Total Cost(FRT_CD)', 'Origin']]
result2.rename(columns={'Origin': 'Freight Factories'}, inplace=True)
                        
grouped_df = production_df.groupby('Unified code')['FactCode'].unique().apply(lambda x: ', '.join(x)).reset_index()
grouped_df.columns = ['Unified code', 'Production Factories']
result_2 = pd.merge(result2, grouped_df, left_on='FG Code', right_on='Unified code')
result_2 = result_2[['Unified code', 'Freight Factories', 'Production Factories']]
result_2['Check'] = (result_2['Freight Factories'] == result_2['Production Factories'])

In [14]:
# Generate result3
df_frt_unique = result_df[['FGSKUCode']].drop_duplicates()
df_DC_unique = demand_df[['FGSKUCode']].drop_duplicates()
df_Production_unique = production_df[['Unified code']].drop_duplicates()

In [15]:
merged_df = pd.merge(df_frt_unique, df_DC_unique, on='FGSKUCode', how='outer', indicator=True)

In [16]:
result3 = pd.DataFrame(columns=['FG Code', 'Availability in Demand + Capacity', 'Availability in Production +Primary Frt'])
for index, row in merged_df.iterrows():
    fg_code = row['FGSKUCode']
    status_dc = 'present' if row['_merge'] == 'both' else 'missing'
    status_production = 'present' if fg_code in df_Production_unique['Unified code'].values else 'missing'
    result3.loc[index] = [fg_code, status_dc, status_production]

In [17]:
#main Block
 
# Write the name of dataframe and tab name that you want in excel
dataframes = {
    'Primary Freight': df_output1,
    'Summary':result_df1,
    'QC_Output1':result_2,
    'QC_Output2':result3
   }
 
# Specify the path to save your Excel file
file_path = r'03-05-2024 Primary Freight.xlsx'
# Using ExcelWriter to write each DataFrame to a separate sheet
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    for sheet_name, dataframe in dataframes.items():
        dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
 
    # Load the workbook and apply formatting
    workbook = writer.book
    for sheet_name in dataframes.keys():
        worksheet = workbook[sheet_name]
        worksheet.sheet_view.showGridLines = False  # Remove gridlines
 
        # Add one row and column at the top
        worksheet.insert_rows(1)
        worksheet.insert_cols(1)
        worksheet.cell(row=1, column=1)
 
        # Add table borders
        border = Border(left=Side(style='thin'), 
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin'))
        for row in worksheet.iter_rows(min_row=2, min_col=2, max_col=worksheet.max_column):
            for cell in row:
                cell.border = border
 
        
# Set light blue color for headers
        for cell in worksheet.iter_cols(min_row=2, min_col=2):
            cell[0].fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue
        # Remove borders for the extra added row and column at the top
        for cell in worksheet['A1:C1'][0]:
            cell.border = None
# Autofit columns
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:  # Necessary to avoid error on empty cells
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[column].width = adjusted_width
# Convert pixels to character units

 
        # Apply auto filters starting from the 2nd row in the B column
        #worksheet.auto_filter.ref = worksheet.dimensions
        last_row = worksheet.max_row
        last_column = worksheet.max_column
        range_str = f"B2:{get_column_letter(last_column)}{last_row}"
        worksheet.auto_filter.ref = range_str
# Save the Excel file
workbook.save(file_path)
 
print("All DataFrames are written to {}".format(file_path))
for table_name in dataframes.keys():
    print(table_name)

All DataFrames are written to 03-05-2024 Primary Freight.xlsx
Primary Freight
Summary
QC_Output1
QC_Output2


In [18]:
# Read the country mapping file
country_mapping_df = pd.read_excel(country_mapping_file)
country_mapping_dict = country_mapping_df.set_index('Country')['Destination Country'].to_dict()

# Extract unique countries from demand_df
unique_countries = demand_df['Country'].unique()

# Create all combinations of SKU, warehouse codes, and countries
skus = demand_df['FGSKUCode'].unique()
warehouses = ['NFCM', 'GFCM', 'KFCM']

# Generate all combinations
combinations = pd.MultiIndex.from_product([skus, warehouses, unique_countries], names=['FGSKUCode', 'Warehouse', 'Country']).to_frame(index=False)

combinations['Origin Country'] = combinations['Warehouse'].map(warehouse_countries)
combinations['Destination Country'] = combinations['Country'].map(country_mapping_dict)

In [19]:
print("Maximum Possible combinations is ",len(skus)*len(unique_countries)*len(warehouses))
print("Number of rows of combinations genertaed is",len(combinations))

Maximum Possible combinations is  36366
Number of rows of combinations genertaed is 36366


In [20]:
#########Refresh Freight Data just to ensure imputation is done correctly
freight_df = pd.read_excel(freight_raw, sheet_name='II. Template - FreightRawData', skiprows=3)
# Drop the first blank column
freight_df = freight_df.drop(columns=['Unnamed: 0'])
freight_df.columns = ['FG Code', 'FG Desc', 'UOM', 'Origin', 'Destination', 'truckload', 'freight_per_truck', 'customs', 'freight_per_uom', 'retain_flag']


# Convert relevant columns to numeric types and calculate cost per carton
freight_df['truckload'] = pd.to_numeric(freight_df['truckload'], errors='coerce')
freight_df['freight_per_truck'] = pd.to_numeric(freight_df['freight_per_truck'], errors='coerce')
freight_df['Total Cost(FRT_CD)'] = freight_df['freight_per_truck'] / freight_df['truckload']

# Add origin country and destination country columns
freight_df['Origin Country'] = freight_df['Origin'].map(factory_country_map)
freight_df['Destination Country'] = freight_df['Destination']

In [21]:
result_df =  pd.merge(combinations, freight_df[['FG Code', 'Origin Country', 'Destination Country', 'Total Cost(FRT_CD)']], left_on=['FGSKUCode', 'Origin Country', 'Destination Country'], right_on=['FG Code', 'Origin Country', 'Destination Country'], how='left')

In [22]:
result_df['Total Cost(FRT_CD)'] = result_df['Total Cost(FRT_CD)'].replace('', np.nan)
result_df['Missing_Flag'] = result_df['Total Cost(FRT_CD)'].isnull().astype(int)
print("Number of missing lanes is",result_df.Missing_Flag.sum())

Number of missing lanes is 18209


In [23]:
result_df['Total Cost(FRT_CD)'] = result_df.apply(replace_with_average, axis=1)

Origin: United Arab Emirates, Destination: Cote d'Ivoire, Origin-Dest Avg Cost: None, Dest Avg Cost: 4.370590301328561
Origin: United Arab Emirates, Destination: Iraq, Origin-Dest Avg Cost: 1.8688926798184797, Dest Avg Cost: 1.4579956015950455
Origin: United Arab Emirates, Destination: Jordan, Origin-Dest Avg Cost: 1.9985340781800958, Dest Avg Cost: 1.0181234087396251
Origin: United Arab Emirates, Destination: Saudi Arabia, Origin-Dest Avg Cost: 1.6639302043710233, Dest Avg Cost: 0.39838039472201886
Origin: United Arab Emirates, Destination: Saudi Arabia, Origin-Dest Avg Cost: 1.6639302043710233, Dest Avg Cost: 0.39838039472201886
Origin: United Arab Emirates, Destination: Kuwait, Origin-Dest Avg Cost: 1.2622971142565018, Dest Avg Cost: 0.5719216205990153
Origin: United Arab Emirates, Destination: Bahrain, Origin-Dest Avg Cost: 1.0606423997099974, Dest Avg Cost: 0.8448386929485593
Origin: United Arab Emirates, Destination: Iraq, Origin-Dest Avg Cost: 1.8688926798184797, Dest Avg Cost: 

Origin: Kuwait, Destination: Bahrain, Origin-Dest Avg Cost: 0.8058173965417722, Dest Avg Cost: 0.8448386929485593
Origin: Kuwait, Destination: Oman, Origin-Dest Avg Cost: 1.0744231953890295, Dest Avg Cost: 1.1240170210458322
Origin: Kuwait, Destination: Qatar, Origin-Dest Avg Cost: 1.1863422782420536, Dest Avg Cost: 1.1110092126448228
Origin: Kuwait, Destination: United Arab Emirates, Origin-Dest Avg Cost: 0.7610497634005627, Dest Avg Cost: 0.9076137073212104
Origin: Kuwait, Destination: Oman, Origin-Dest Avg Cost: 1.0744231953890295, Dest Avg Cost: 1.1240170210458322
Origin: Kuwait, Destination: Qatar, Origin-Dest Avg Cost: 1.1863422782420536, Dest Avg Cost: 1.1110092126448228
Origin: Kuwait, Destination: United Arab Emirates, Origin-Dest Avg Cost: 0.7610497634005627, Dest Avg Cost: 0.9076137073212104
Origin: Saudi Arabia, Destination: Cote d'Ivoire, Origin-Dest Avg Cost: 4.304225273223811, Dest Avg Cost: 4.370590301328561
Origin: Saudi Arabia, Destination: Iraq, Origin-Dest Avg Cost:

In [24]:
print("Number of rows of combinations genertaed for secondary freight is",len(result_df))

Number of rows of combinations genertaed for secondary freight is 36366


In [25]:
print("Number of missing values ",len(result_df[result_df['Total Cost(FRT_CD)']=='']))

Number of missing values  0


In [26]:
# Keep and rename the specified columns
result_df = result_df[['FGSKUCode','Warehouse', 'Country','Missing_Flag','Total Cost(FRT_CD)']]
result_df['retain_flag']=1
result_df.head()

,FGSKUCode,Warehouse,Country,Missing_Flag,Total Cost(FRT_CD),retain_flag
0,4001351001,NFCM,IS & PL_Cote d'Ivoire,0,2.794805,1
1,4001351001,NFCM,IS & PL_Iraq,0,1.089206,1
2,4001351001,NFCM,IS & PL_Jordan,0,0.687706,1
3,4001351001,NFCM,IS & PL_KSA,0,0.000000,1
4,4001351001,NFCM,KSA_KSA,0,0.000000,1


In [27]:
#############Secondary Freight Output Generation
df_final = result_df
# Get the number of distinct values for each column
distinct_counts = df_final.nunique()

# Get the number of null values for each column
null_counts = df_final.isnull().sum()
counts=df_final.count()
# Get the count, mean, std, min, and percentile distribution for each column
summary_stats = df_final.describe(percentiles=[0.25, 0.5, 0.75]).transpose()

# Combine all the results into a single DataFrame
result_df1 = pd.DataFrame({
    'Count':counts,
    'Distinct Values': distinct_counts,
    'Null Count': null_counts,
    'Mean': summary_stats['mean'],
    'Std': summary_stats['std'],
    'Minimum Value': summary_stats['min'],
    'Percentile Distribution (25%)': summary_stats['25%'],
    'Percentile Distribution (50%)': summary_stats['50%'],
    'Percentile Distribution (75%)': summary_stats['75%'],
    'Maximum Value': summary_stats['max']
})


result_df1=result_df1.copy()
result_df1['Column Name']=result_df1.index
result_df1=result_df1[['Column Name','Count', 'Distinct Values', 'Null Count', 'Mean', 'Std', 'Minimum Value', 'Percentile Distribution (25%)',
       'Percentile Distribution (50%)', 'Percentile Distribution (75%)', 'Maximum Value']]

In [28]:
#######################Secondary Freight Output Writing Code
# Write the name of dataframe and tab name that you want in excel
dataframes = {
    'Secondary Freight': result_df,
    'Summary':result_df1,
   }
 
# Specify the path to save your Excel file
file_path = r'03-05-2024 Secondary Freight.xlsx'
# Using ExcelWriter to write each DataFrame to a separate sheet
with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    for sheet_name, dataframe in dataframes.items():
        dataframe.to_excel(writer, sheet_name=sheet_name, index=False)
 
    # Load the workbook and apply formatting
    workbook = writer.book
    for sheet_name in dataframes.keys():
        worksheet = workbook[sheet_name]
        worksheet.sheet_view.showGridLines = False  # Remove gridlines
 
        # Add one row and column at the top
        worksheet.insert_rows(1)
        worksheet.insert_cols(1)
        worksheet.cell(row=1, column=1)
 
        # Add table borders
        border = Border(left=Side(style='thin'), 
                        right=Side(style='thin'), 
                        top=Side(style='thin'), 
                        bottom=Side(style='thin'))
        for row in worksheet.iter_rows(min_row=2, min_col=2, max_col=worksheet.max_column):
            for cell in row:
                cell.border = border
 
        
# Set light blue color for headers
        for cell in worksheet.iter_cols(min_row=2, min_col=2):
            cell[0].fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue
        # Remove borders for the extra added row and column at the top
        for cell in worksheet['A1:C1'][0]:
            cell.border = None
# Autofit columns
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:  # Necessary to avoid error on empty cells
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[column].width = adjusted_width
# Convert pixels to character units

 
        # Apply auto filters starting from the 2nd row in the B column
        #worksheet.auto_filter.ref = worksheet.dimensions
        last_row = worksheet.max_row
        last_column = worksheet.max_column
        range_str = f"B2:{get_column_letter(last_column)}{last_row}"
        worksheet.auto_filter.ref = range_str
# Save the Excel file
workbook.save(file_path)
 
print("All DataFrames are written to {}".format(file_path))
for table_name in dataframes.keys():
    print(table_name)

All DataFrames are written to 03-05-2024 Secondary Freight.xlsx
Secondary Freight
Summary
